In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


# !cp "/content/gdrive/My Drive/DataAugumentation.zip" .
#ADDED NEW SOURCE
!cp "/content/gdrive/My Drive/data_aug(3_slices_with_repeated)_cluster_5.zip" .
!unzip -qq DataAugumentation.zip
!unzip -qq data_aug_3_slices_with_repeated_cluster_5.zip
!rm DataAugumentation.zip
!rm data_aug(3_slices_with_repeated)_cluster_5.zip
data_path = 'DataAugumentation'

Mounted at /content/gdrive
unzip:  cannot find or open DataAugumentation.zip, DataAugumentation.zip.zip or DataAugumentation.zip.ZIP.
unzip:  cannot find or open data_aug_3_slices_with_repeated_cluster_5.zip, data_aug_3_slices_with_repeated_cluster_5.zip.zip or data_aug_3_slices_with_repeated_cluster_5.zip.ZIP.
rm: cannot remove 'DataAugumentation.zip': No such file or directory
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `rm data_aug(3_slices_with_repeated)_cluster_5.zip'


In [4]:
!unzip -qq data_aug_3_slices_with_repeated_cluster_5.zip

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import joblib

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 1e-4
PLOT_SAVE_DIR = 'predicted_vs_actual_plots'


def process_file(file_path):
    df = pd.read_csv(file_path)

    df['Time'] = df['Time'] - df['Time'].iloc[0]

    df['Momentary fuel consumption'] = df['Trip fuel consumption'].diff().fillna(0)

    features = df[['Engine speed', 'Throttle position', 'Accelerator pedal position', 'Speed']]
    target = df['Momentary fuel consumption']

    features = features.iloc[:SEQUENCE_LENGTH]
    target = target.iloc[:SEQUENCE_LENGTH]

    return features.values, target.values



def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    return normalized_data


scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_original = []
y_original = []
X_augmented = []
y_augmented = []

base_folder_path = '/content/'

# CHANGED TO 6 FOR NEW DATA
for i in range(6):
  if i == 5:
    folder_path = os.path.join(base_folder_path, f'data_aug(3_slices_with_repeated)_cluster_{i}')
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            features, target = process_file(file_path)

            slices = filename.split('_')
            is_original_trip = slices[0] == slices[2] and slices[0] == slices[4]

            if is_original_trip:
                X_original.append(features)
                y_original.append(target)
            else:
                X_augmented.append(features)
                y_augmented.append(target)

# Pad and convert lists to numpy arrays
X_original = pad_sequences(X_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_original = pad_sequences(y_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
X_augmented = pad_sequences(X_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_augmented = pad_sequences(y_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')

num_test = int(0.2 * len(X_original))
X_test = X_original[:num_test]
y_test = y_original[:num_test]
X_train = np.concatenate([X_original[num_test:], X_augmented])
y_train = np.concatenate([y_original[num_test:], y_augmented])

scaler_X.fit(X_train.reshape(-1, X_train.shape[-1]))
scaler_y.fit(y_train.reshape(-1, 1))

joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']

In [6]:
X_train_normalized = pad_and_normalize(X_train, scaler_X)
y_train_normalized = scaler_y.transform(y_train.reshape(-1, 1)).reshape(y_train.shape)
X_test_normalized = pad_and_normalize(X_test, scaler_X)
y_test_normalized = scaler_y.transform(y_test.reshape(-1, 1)).reshape(y_test.shape)

print('X_train_normalized shape:', X_train_normalized.shape)
print('y_train_normalized shape:', y_train_normalized.shape)
print('X_test_normalized shape:', X_test_normalized.shape)
print('y_test_normalized shape:', y_test_normalized.shape)


X_train_normalized shape: (5829, 600, 4)
y_train_normalized shape: (5829, 600)
X_test_normalized shape: (3, 600, 4)
y_test_normalized shape: (3, 600)


In [7]:
model = Sequential([
    Bidirectional(LSTM(32, return_sequences=True, input_shape=(SEQUENCE_LENGTH, X_train_normalized.shape[-1]))),
    Dropout(0.2),
    Bidirectional(LSTM(32, return_sequences=True)),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')

model.fit(X_train_normalized, y_train_normalized, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 15s 101ms/step - loss: 0.1148 - val_loss: 0.0116
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 15s 88ms/step - loss: 0.0162 - val_loss: 0.0092
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - loss: 0.0129 - val_loss: 0.0074
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - loss: 0.0102 - val_loss: 0.0063
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - loss: 0.0087 - val_loss: 0.0057
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0077 - val_loss: 0.0053
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - loss: 0.0072 - val_loss: 0.0051
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - loss: 0.0066 - val_loss: 0.0049
Epoch 9/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - loss: 0.0063 - val_loss: 0.0047
Epoch 10/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - loss: 0.0060 - val_loss: 0.0046
Epoch 11/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - loss: 0.0057 - val_loss: 0.0044
Epoch 12/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - 

In [8]:
model.save('modelBLSTM_new_majid_added.h5')

In [9]:
model.save('/content/drive/My Drive/model.h5')

In [10]:
test_loss = model.evaluate(X_test_normalized, y_test_normalized)
print(f"Test Loss: {test_loss}")

y_pred_normalized = model.predict(X_test_normalized)

y_test_inv = scaler_y.inverse_transform(y_test_normalized.reshape(-1, SEQUENCE_LENGTH))
y_pred_inv = scaler_y.inverse_transform(y_pred_normalized.reshape(-1, SEQUENCE_LENGTH))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0033
Test Loss: 0.0033266451209783554
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step


In [11]:
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 366013.15625


In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt

if not os.path.exists(PLOT_SAVE_DIR):
    os.makedirs(PLOT_SAVE_DIR)

for i in range(len(X_test)):
    # Convert momentary to cumulative
    cumulative_actuals = np.cumsum(y_test_inv[i], axis=0)  # assuming y_test_inv is already the inverse transformed data
    cumulative_predictions = np.cumsum(y_pred_inv[i], axis=0)  # assuming y_pred_inv is already the inverse transformed predictions

    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_actuals, label='Actual Trip Fuel Consumption')
    plt.plot(cumulative_predictions, label='Predicted Trip Fuel Consumption')
    plt.title(f'Trip {i + 1}: Actual vs Predicted Fuel Consumption')
    plt.xlabel('Time Steps')
    plt.ylabel('Trip Fuel Consumption (uL)')
    plt.legend()

    plot_filename = os.path.join(PLOT_SAVE_DIR, f'trip_{i + 1}_actual_vs_predicted.png')
    plt.savefig(plot_filename)
    plt.close()  # Close the plot to save memory



In [13]:
!zip -r data.zip predicted_vs_actual_plots

  adding: predicted_vs_actual_plots/ (stored 0%)
  adding: predicted_vs_actual_plots/trip_3_actual_vs_predicted.png (deflated 10%)
  adding: predicted_vs_actual_plots/trip_1_actual_vs_predicted.png (deflated 10%)
  adding: predicted_vs_actual_plots/trip_2_actual_vs_predicted.png (deflated 9%)
